In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [9]:
import zipfile
import os

zip_path = "/content/archive (1).zip"
extract_path = "/content/fer_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

os.listdir(extract_path)



['test', 'train']

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/fer_data/train"
test_dir = "/content/fer_data/test"

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical'
)



Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [11]:
train_generator.class_indices


{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,BatchNormalization

model=Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(48,48,1)))
BatchNormalization()
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
BatchNormalization()
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
BatchNormalization()
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Conv2D(256,(3,3),activation='relu'))
BatchNormalization()
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256,activation='relu'))

model.add(Dense(7,activation='softmax'))

model.summary()




Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 2, 2, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 455,431 (1.74 MB)

 Trainable params: 455,431 (1.74 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [14]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history=model.fit(train_generator,epochs=10,validation_data=test_generator,callbacks=[early_stop])

Epoch 1/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 143s 309ms/step - accuracy: 0.2501 - loss: 1.8231 - val_accuracy: 0.2942 - val_loss: 1.7256
Epoch 2/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 142s 316ms/step - accuracy: 0.3284 - loss: 1.6751 - val_accuracy: 0.4305 - val_loss: 1.4744
Epoch 3/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 138s 307ms/step - accuracy: 0.4261 - loss: 1.4749 - val_accuracy: 0.4836 - val_loss: 1.3404
Epoch 4/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 149s 332ms/step - accuracy: 0.4641 - loss: 1.3722 - val_accuracy: 0.5089 - val_loss: 1.2882
Epoch 5/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 142s 317ms/step - accuracy: 0.5015 - loss: 1.3151 - val_accuracy: 0.5227 - val_loss: 1.2230
Epoch 6/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 138s 307ms/step - accuracy: 0.5194 - loss: 1.2542 - val_accuracy: 0.5454 - val_loss: 1.1886
Epoch 7/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 141s 314ms/step - accuracy: 0.5296 - loss: 1.2303 - val_accuracy: 0.5513 - val_loss: 1.1699
Epoch 8/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 137s 304ms/step - accuracy: 0.5405 -

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_gen = ImageDataGenerator(rescale=1./255)

train_generator_aug = train_aug.flow_from_directory(
    "/content/fer_data/train",
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical'
)

test_generator = test_gen.flow_from_directory(
    "/content/fer_data/test",
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical'
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [16]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history=model.fit(train_generator_aug,epochs=50,validation_data=test_generator,callbacks=[early_stop])

Epoch 1/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 159s 354ms/step - accuracy: 0.4041 - loss: 1.5295 - val_accuracy: 0.5492 - val_loss: 1.1770
Epoch 2/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 160s 357ms/step - accuracy: 0.4248 - loss: 1.4728 - val_accuracy: 0.5419 - val_loss: 1.1903
Epoch 3/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 154s 343ms/step - accuracy: 0.4414 - loss: 1.4520 - val_accuracy: 0.5471 - val_loss: 1.1760
Epoch 4/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 161s 357ms/step - accuracy: 0.4463 - loss: 1.4307 - val_accuracy: 0.5386 - val_loss: 1.1783
Epoch 5/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 153s 340ms/step - accuracy: 0.4504 - loss: 1.4268 - val_accuracy: 0.5607 - val_loss: 1.1533
Epoch 6/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 151s 336ms/step - accuracy: 0.4579 - loss: 1.4085 - val_accuracy: 0.5482 - val_loss: 1.1788
Epoch 7/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 154s 342ms/step - accuracy: 0.4558 - loss: 1.4129 - val_accuracy: 0.5638 - val_loss: 1.1411
Epoch 8/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 160s 355ms/step - accuracy: 0.4608 -